In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm as tq
import time
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import ArcGIS
df_health = pd.read_csv("healthcare_noshows.csv")

df_health



PatientId  AppointmentID Gender ScheduledDay AppointmentDay  Age  \
0       2.987250e+13        5642903      F   2016-04-29     2016-04-29   62   
1       5.589978e+14        5642503      M   2016-04-29     2016-04-29   56   
2       4.262962e+12        5642549      F   2016-04-29     2016-04-29   62   
3       8.679512e+11        5642828      F   2016-04-29     2016-04-29    8   
4       8.841186e+12        5642494      F   2016-04-29     2016-04-29   56   
...              ...            ...    ...          ...            ...  ...   
106982  2.572134e+12        5651768      F   2016-05-03     2016-06-07   56   
106983  3.596266e+12        5650093      F   2016-05-03     2016-06-07   51   
106984  1.557663e+13        5630692      F   2016-04-27     2016-06-07   21   
106985  9.213493e+13        5630323      F   2016-04-27     2016-06-07   38   
106986  3.775115e+14        5629448      F   2016-04-27     2016-06-07   54   

            Neighbourhood  Scholarship  Hipertension  Diabetes  Alcoholism  \
0         JARDIM DA PENHA        False          True     False       False   
1         JARDIM DA PENHA        False         False     False       False   
2           MATA DA PRAIA        False         False     False       False   
3       PONTAL DE CAMBURI        False         False     False       False   
4         JARDIM DA PENHA        False          True      True       False   
...                   ...          ...           ...       ...         ...   
106982        MARIA ORTIZ        False         False     False       False   
106983        MARIA ORTIZ        False         False     False       False   
106984        MARIA ORTIZ        False         False     False       False   
106985        MARIA ORTIZ        False         False     False       False   
106986        MARIA ORTIZ        False         False     False       False   

        Handcap  SMS_received  Showed_up  Date.diff  
0         False         False       True          0  
1         False         False       True          0  
2         False         False       True          0  
3         False         False       True          0  
4         False         False       True          0  
...         ...           ...        ...        ...  
106982    False          True       True         35  
106983    False          True       True         35  
106984    False          True       True         41  
106985    False          True       True         41  
106986    False          True       True         41  

[106987 rows x 15 columns]

In [128]:
%pip install tqdm



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [129]:
df_health["Age"].value_counts()

Age
1      2273
52     1746
49     1652
53     1651
56     1635
       ... 
98        6
115       5
100       4
102       2
99        1
Name: count, Length: 102, dtype: int64

In [130]:
df_health = df_health.drop(columns = ["PatientId", "AppointmentID", "ScheduledDay", "AppointmentDay", "Date.diff"])

In [131]:
df_health

Gender  Age      Neighbourhood  Scholarship  Hipertension  Diabetes  \
0           F   62    JARDIM DA PENHA        False          True     False   
1           M   56    JARDIM DA PENHA        False         False     False   
2           F   62      MATA DA PRAIA        False         False     False   
3           F    8  PONTAL DE CAMBURI        False         False     False   
4           F   56    JARDIM DA PENHA        False          True      True   
...       ...  ...                ...          ...           ...       ...   
106982      F   56        MARIA ORTIZ        False         False     False   
106983      F   51        MARIA ORTIZ        False         False     False   
106984      F   21        MARIA ORTIZ        False         False     False   
106985      F   38        MARIA ORTIZ        False         False     False   
106986      F   54        MARIA ORTIZ        False         False     False   

        Alcoholism  Handcap  SMS_received  Showed_up  
0            False    False         False       True  
1            False    False         False       True  
2            False    False         False       True  
3            False    False         False       True  
4            False    False         False       True  
...            ...      ...           ...        ...  
106982       False    False          True       True  
106983       False    False          True       True  
106984       False    False          True       True  
106985       False    False          True       True  
106986       False    False          True       True  

[106987 rows x 10 columns]

In [132]:
def preprocessing(df_health):
    df_health["Gender"] = df_health["Gender"].apply(lambda x: 1 if x == "M" else 2)
    df_health["Scholarship"] = df_health["Scholarship"].apply(lambda x: 0 if x == "False" else 1)
    df_health["Hipertension"] = df_health["Hipertension"].apply(lambda x: 0 if x == "False" else 1)
    df_health["Diabetes"] = df_health["Diabetes"].apply(lambda x: 0 if x == "False" else 1)
    df_health["Alcoholism"] = df_health["Alcoholism"].apply(lambda x: 0 if x == "False" else 1)
    df_health["Handcap"] = df_health["Handcap"].apply(lambda x: 0 if x == "False" else 1)
    df_health["SMS_received"] = df_health["SMS_received"].apply(lambda x: 0 if x == "False" else 1)
    df_health["Showed_up"] = df_health["Showed_up"].apply(lambda x: 0 if x == "False" else 1)
    return df_health

In [133]:
preprocessing(df_health)

Gender  Age      Neighbourhood  Scholarship  Hipertension  Diabetes  \
0            2   62    JARDIM DA PENHA            1             1         1   
1            1   56    JARDIM DA PENHA            1             1         1   
2            2   62      MATA DA PRAIA            1             1         1   
3            2    8  PONTAL DE CAMBURI            1             1         1   
4            2   56    JARDIM DA PENHA            1             1         1   
...        ...  ...                ...          ...           ...       ...   
106982       2   56        MARIA ORTIZ            1             1         1   
106983       2   51        MARIA ORTIZ            1             1         1   
106984       2   21        MARIA ORTIZ            1             1         1   
106985       2   38        MARIA ORTIZ            1             1         1   
106986       2   54        MARIA ORTIZ            1             1         1   

        Alcoholism  Handcap  SMS_received  Showed_up  
0                1        1             1          1  
1                1        1             1          1  
2                1        1             1          1  
3                1        1             1          1  
4                1        1             1          1  
...            ...      ...           ...        ...  
106982           1        1             1          1  
106983           1        1             1          1  
106984           1        1             1          1  
106985           1        1             1          1  
106986           1        1             1          1  

[106987 rows x 10 columns]

In [134]:
preprocessing(df_health)

Gender  Age      Neighbourhood  Scholarship  Hipertension  Diabetes  \
0            2   62    JARDIM DA PENHA            1             1         1   
1            2   56    JARDIM DA PENHA            1             1         1   
2            2   62      MATA DA PRAIA            1             1         1   
3            2    8  PONTAL DE CAMBURI            1             1         1   
4            2   56    JARDIM DA PENHA            1             1         1   
...        ...  ...                ...          ...           ...       ...   
106982       2   56        MARIA ORTIZ            1             1         1   
106983       2   51        MARIA ORTIZ            1             1         1   
106984       2   21        MARIA ORTIZ            1             1         1   
106985       2   38        MARIA ORTIZ            1             1         1   
106986       2   54        MARIA ORTIZ            1             1         1   

        Alcoholism  Handcap  SMS_received  Showed_up  
0                1        1             1          1  
1                1        1             1          1  
2                1        1             1          1  
3                1        1             1          1  
4                1        1             1          1  
...            ...      ...           ...        ...  
106982           1        1             1          1  
106983           1        1             1          1  
106984           1        1             1          1  
106985           1        1             1          1  
106986           1        1             1          1  

[106987 rows x 10 columns]

In [135]:
df_sampled = preprocessing(df_health)
df_sampled = df_health.sample(n=20)

In [136]:
df_sampled

Gender  Age   Neighbourhood  Scholarship  Hipertension  Diabetes  \
31420       2   22    SOLON BORGES            1             1         1   
86263       2   17     JOANA D´ARC            1             1         1   
5491        2    7  JARDIM CAMBURI            1             1         1   
53381       2   71  NOVA PALESTINA            1             1         1   
75309       2   58          BONFIM            1             1         1   
83262       2   55  JARDIM CAMBURI            1             1         1   
17537       2   63     JOANA D´ARC            1             1         1   
60269       2   22    SOLON BORGES            1             1         1   
54549       2   30         ITARARÉ            1             1         1   
33546       2   30        REDENÇÃO            1             1         1   
59660       2   34    SOLON BORGES            1             1         1   
4704        2   66   SANTO ANTÔNIO            1             1         1   
3016        2   84         ITARARÉ            1             1         1   
49081       2   52     SANTOS REIS            1             1         1   
7182        2   56        GURIGICA            1             1         1   
61691       2   12        SÃO JOSÉ            1             1         1   
82948       2   38  JARDIM CAMBURI            1             1         1   
23538       2   24        SÃO JOSÉ            1             1         1   
72665       2   57     SANTO ANDRÉ            1             1         1   
44083       2   15  PARQUE MOSCOSO            1             1         1   

       Alcoholism  Handcap  SMS_received  Showed_up  
31420           1        1             1          1  
86263           1        1             1          1  
5491            1        1             1          1  
53381           1        1             1          1  
75309           1        1             1          1  
83262           1        1             1          1  
17537           1        1             1          1  
60269           1        1             1          1  
54549           1        1             1          1  
33546           1        1             1          1  
59660           1        1             1          1  
4704            1        1             1          1  
3016            1        1             1          1  
49081           1        1             1          1  
7182            1        1             1          1  
61691           1        1             1          1  
82948           1        1             1          1  
23538           1        1             1          1  
72665           1        1             1          1  
44083           1        1             1          1

In [137]:
# df_sampled["Latitude"] = df_sampled.apply(lambda x: "", axis = 1)
# df_sampled["Longitude"] = df_sampled.apply(lambda x: "", axis = 1)

df_sampled["Latitude"] = 0.00
df_sampled["Longitude"] = 0.00

In [138]:
df_sampled

Gender  Age   Neighbourhood  Scholarship  Hipertension  Diabetes  \
31420       2   22    SOLON BORGES            1             1         1   
86263       2   17     JOANA D´ARC            1             1         1   
5491        2    7  JARDIM CAMBURI            1             1         1   
53381       2   71  NOVA PALESTINA            1             1         1   
75309       2   58          BONFIM            1             1         1   
83262       2   55  JARDIM CAMBURI            1             1         1   
17537       2   63     JOANA D´ARC            1             1         1   
60269       2   22    SOLON BORGES            1             1         1   
54549       2   30         ITARARÉ            1             1         1   
33546       2   30        REDENÇÃO            1             1         1   
59660       2   34    SOLON BORGES            1             1         1   
4704        2   66   SANTO ANTÔNIO            1             1         1   
3016        2   84         ITARARÉ            1             1         1   
49081       2   52     SANTOS REIS            1             1         1   
7182        2   56        GURIGICA            1             1         1   
61691       2   12        SÃO JOSÉ            1             1         1   
82948       2   38  JARDIM CAMBURI            1             1         1   
23538       2   24        SÃO JOSÉ            1             1         1   
72665       2   57     SANTO ANDRÉ            1             1         1   
44083       2   15  PARQUE MOSCOSO            1             1         1   

       Alcoholism  Handcap  SMS_received  Showed_up  Latitude  Longitude  
31420           1        1             1          1       0.0        0.0  
86263           1        1             1          1       0.0        0.0  
5491            1        1             1          1       0.0        0.0  
53381           1        1             1          1       0.0        0.0  
75309           1        1             1          1       0.0        0.0  
83262           1        1             1          1       0.0        0.0  
17537           1        1             1          1       0.0        0.0  
60269           1        1             1          1       0.0        0.0  
54549           1        1             1          1       0.0        0.0  
33546           1        1             1          1       0.0        0.0  
59660           1        1             1          1       0.0        0.0  
4704            1        1             1          1       0.0        0.0  
3016            1        1             1          1       0.0        0.0  
49081           1        1             1          1       0.0        0.0  
7182            1        1             1          1       0.0        0.0  
61691           1        1             1          1       0.0        0.0  
82948           1        1             1          1       0.0        0.0  
23538           1        1             1          1       0.0        0.0  
72665           1        1             1          1       0.0        0.0  
44083           1        1             1          1       0.0        0.0

In [151]:

location = df_sampled["Neighbourhood"]
geolocator = Nominatim(user_agent="my_geocoder")
gecoding = ArcGIS(scheme="https")
def geocoder(location):
    geo = geolocator.geocode(location)
    if geo:
        return geo.latitude, geo.longitude
    else:
        return None, None


for i in range(len(df_sampled.index)):
    long,lat = geocoder(df_sampled['Neighbourhood'].iloc[[i]])
    df_sampled["Longitude"][i] = long
    df_sampled["Latitude"][i] = lat

    df_sampled.loc[i, "Longitude"] = long
    df_sampled.loc[i, "Latitude"] = lat
    print(df_sampled)
# df_sampled[i, "Longitutde"] = long
# df_sampled[i, "Latitude"] = lat




/var/folders/z1/rjx7d_zx5yb0cfkp8qbb5d9w0000gn/T/ipykernel_60629/908226663.py:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_sampled["Longitude"][i] = long
/var/folders/z1/rjx7d_zx5yb0cfkp8qbb5d9w0000gn/T/ipykernel_60629/908226663.py:1

       Gender   Age   Neighbourhood  Scholarship  Hipertension  Diabetes  \
31420     2.0  22.0    SOLON BORGES          1.0           1.0       1.0   
86263     2.0  17.0     JOANA D´ARC          1.0           1.0       1.0   
5491      2.0   7.0  JARDIM CAMBURI          1.0           1.0       1.0   
53381     2.0  71.0  NOVA PALESTINA          1.0           1.0       1.0   
75309     2.0  58.0          BONFIM          1.0           1.0       1.0   
83262     2.0  55.0  JARDIM CAMBURI          1.0           1.0       1.0   
17537     2.0  63.0     JOANA D´ARC          1.0           1.0       1.0   
60269     2.0  22.0    SOLON BORGES          1.0           1.0       1.0   
54549     2.0  30.0         ITARARÉ          1.0           1.0       1.0   
33546     2.0  30.0        REDENÇÃO          1.0           1.0       1.0   
59660     2.0  34.0    SOLON BORGES          1.0           1.0       1.0   
4704      2.0  66.0   SANTO ANTÔNIO          1.0           1.0       1.0   
3016      2.

/var/folders/z1/rjx7d_zx5yb0cfkp8qbb5d9w0000gn/T/ipykernel_60629/908226663.py:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_sampled["Longitude"][i] = long
/var/folders/z1/rjx7d_zx5yb0cfkp8qbb5d9w0000gn/T/ipykernel_60629/908226663.py:1

       Gender   Age   Neighbourhood  Scholarship  Hipertension  Diabetes  \
31420     2.0  22.0    SOLON BORGES          1.0           1.0       1.0   
86263     2.0  17.0     JOANA D´ARC          1.0           1.0       1.0   
5491      2.0   7.0  JARDIM CAMBURI          1.0           1.0       1.0   
53381     2.0  71.0  NOVA PALESTINA          1.0           1.0       1.0   
75309     2.0  58.0          BONFIM          1.0           1.0       1.0   
83262     2.0  55.0  JARDIM CAMBURI          1.0           1.0       1.0   
17537     2.0  63.0     JOANA D´ARC          1.0           1.0       1.0   
60269     2.0  22.0    SOLON BORGES          1.0           1.0       1.0   
54549     2.0  30.0         ITARARÉ          1.0           1.0       1.0   
33546     2.0  30.0        REDENÇÃO          1.0           1.0       1.0   
59660     2.0  34.0    SOLON BORGES          1.0           1.0       1.0   
4704      2.0  66.0   SANTO ANTÔNIO          1.0           1.0       1.0   
3016      2.

GeocoderTimedOut: Service timed out

In [ ]:

# location = "JARDIM DA PENHA"
# geolocator = Nominatim(user_agent="my_geocoder")
# gecoding = ArcGIS(scheme="https")
# def geocoder(location):
#     geo = geolocator.geocode(location)
#     if geo:
#         return geo.latitude, geo.longitude
#     else:
#         return None, None


# for i in range(len(df_sampled.index)):
#     long,lat = geocoder(df_sampled['Neighbourhood'].iloc[[i]])
#     #df_sampled["Longitude"][i] = long
#     #df_sampled["Latitude"][i] = lat

#     df_sampled.loc[i, "Longitude"] = geocoder(location)
#     df_sampled.loc[i, "Latitude"] = geocoder(location)
# # df_sampled[i, "Longitutde"] = long
# # df_sampled[i, "Latitude"] = lat




In [ ]:
len(df_sampled.index)

60

In [ ]:
geocoder(location)

(None, None)

In [ ]:
# location = "JARDIM DA PENHA"
# geolocator = Nominatim(user_agent="health_geolocating")
# geo = ArcGIS(sche)
# def geocoder(location):
#     in_array = location
#     out_list = []
#     for i in tq(in_array):
#         out_list.append(geolocator.geocode(i))
#     return out_list
#         #geo = geolocator.geocode(location)
#     #if geo:
#     #    return out_list
#     #else:
#     #    return None, None






In [ ]:
print(df_sampled)

       Gender   Age   Neighbourhood  Scholarship  Hipertension  Diabetes  \
31420     2.0  22.0    SOLON BORGES          1.0           1.0       1.0   
86263     2.0  17.0     JOANA D´ARC          1.0           1.0       1.0   
5491      2.0   7.0  JARDIM CAMBURI          1.0           1.0       1.0   
53381     2.0  71.0  NOVA PALESTINA          1.0           1.0       1.0   
75309     2.0  58.0          BONFIM          1.0           1.0       1.0   
83262     2.0  55.0  JARDIM CAMBURI          1.0           1.0       1.0   
17537     2.0  63.0     JOANA D´ARC          1.0           1.0       1.0   
60269     2.0  22.0    SOLON BORGES          1.0           1.0       1.0   
54549     2.0  30.0         ITARARÉ          1.0           1.0       1.0   
33546     2.0  30.0        REDENÇÃO          1.0           1.0       1.0   
59660     2.0  34.0    SOLON BORGES          1.0           1.0       1.0   
4704      2.0  66.0   SANTO ANTÔNIO          1.0           1.0       1.0   
3016      2.

In [ ]:
# thing = geocoder(location)
# print(thing)

In [ ]:
# def do_geocode(location, attempt = 1, max_attempts = 5):
#     try:
#         return geopy.gecode(location)
#     except GeocoderTimedOut:
#         if attempt <= max_attemps:
#             return do_geocode(location, attempt = attempt + 1)
#         raise

In [ ]:
df_sampled["Latitude, Longitutde"] = df_sampled["Neighbourhood"].apply(geocoder(location))


TypeError: 'NoneType' object is not callable

In [ ]:
X = df_health.drop("Alcoholism", axis = 1)
y = df_health["Alcoholism"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [ ]:
ros = RandomOverSampler(random_state = 42)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [ ]:
model = LogisticRegression(random_state=42)
model.fit(X_resampled, y_resampled)

In [ ]:
df_sampled.to_csv("Cleaned.csv", index=False)